Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `???`, `YOUR CODE HERE`, "???", "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

$\newcommand{\TirName}[1]{\text{#1}}
\newcommand{\inferrule}[3][]{
  \let\and\qquad
  \begin{array}{@{}l@{}}
  \TirName{#1}
  \\
  \displaystyle
  \frac{#2}{#3}
  \end{array}
}
\newcommand{\infer}[3][]{\inferrule[#1]{#2}{#3}}
$

# Exercise: Programming with Encapsulated Effects

<!-- 3 Expressions -->

<!-- 4 Binding and Scope -->

<!-- 8 Recursion -->

<!-- 9 Inductive Data Types -->

<!-- 11 Concrete Syntax -->

<!-- 12 Abstract Syntax and Parsing -->

<!-- 13 Exercise: Syntax -->

<!-- 14 Static Scoping -->

<!-- 15 Judgments -->

<!-- 16 Variables, Basic Values, and Judgments Lab -->

<!-- 18 Operational Semantics -->

<!-- 19 Functions and Dynamic Scoping -->

<!-- 20 Big-Step Exercise -->

<!-- 21 Evaluation Order  -->

<!-- 26 Static Type Checking -->

<!-- 27 Objects -->

<!-- 27 Static Type Checking Lab -->

<!-- 30 Mutable State -->

<!-- Encapsulating Effects Exercise -->

### Learning Goals

The primary learning goal of this exercise is to get experience
programming with encapsulated effects.

We will also consider the idea of transforming code represented as an
abstract syntax tree to make it easier to implement subsequent passes
like interpretation.

### Instructions

This assignment asks you to write Scala code. There are restrictions
associated with how you can solve these problems. Please pay careful
heed to those. If you are unsure, ask the course staff.

Note that `???` indicates that there is a missing function or code
fragment that needs to be filled in. Make sure that you remove the `???`
and replace it with the answer.

Use the test cases provided to test your implementations. You are also
encouraged to write your own test cases to help debug your work.
However, please delete any extra cells you may have created lest they
break an autograder.

### Imports

In [ ]:
// Run this cell FIRST before testing.
import $ivy.`org.scalatest::scalatest:3.2.19`, org.scalatest._, events._, flatspec._, matchers.should._
import $ivy.`org.scalatestplus::scalacheck-1-18:3.2.19.0`, org.scalatestplus.scalacheck._
def report(suite: Suite): Unit = suite.execute(stats = true)
def assertPassed(suite: Suite): Unit =
  suite.run(None, Args(new Reporter {
    def apply(e: Event) = e match {
      case e @ (_: TestFailed) => assert(false, s"${e.message} (${e.testName})")
      case _ => ()
    }
  }))
def passed(points: Int): Unit = {
  require(points >= 0)
  if (points == 1) println("*** 🎉 Tests Passed (1 point) ***")
  else println(s"*** 🎉 Tests Passed ($points points) ***")
}
def test(suite: Suite, points: Int): Unit = {
  report(suite)
  assertPassed(suite)
  passed(points)
}

## TypeScripty: Numbers, Booleans, and Functions

### Syntax

In this assignment, we consider a simplified TypeScripty with numbers,
booleans, and functions types.

Function literals
$x\texttt{(}y\texttt{:}\,\tau\texttt{)}\texttt{:}\,\tau' \mathrel{\texttt{=}\!\texttt{>}} e_1$
have exactly one parameter, are always named, and must have a return
type annotation. The name may be used to define recursive functions.

The expressions include variable uses $x$, variable binding
$\mathbf{const}\;x\;\texttt{=}\;e_1\texttt{;}\;e_2$, unary
$\mathop{\mathit{uop}} e_1$ and binary $e_1 \mathbin{\mathit{bop}} e_2$
expressions, if-then-else $e_1\;\texttt{?}\;e_2\;\texttt{:}\;e_3$, and
function call $e_1\texttt{(}e_2\texttt{)}$. The unary operators are
limited to number negation $\texttt{-}$ and boolean negation
$\texttt{!}$, and the binary operators are limited to number addition
$\texttt{+}$, number times $\texttt{*}$, and equality $\texttt{===}$:

$$
\begin{array}{rrrl}
  \text{types} & \tau& \mathrel{::=}&
  \texttt{number}
  \mid\texttt{bool}
  \mid\texttt{(}y\texttt{:}\,\tau\texttt{)} \mathrel{\texttt{=}\!\texttt{>}} \tau'
  \\
  \text{values} & v& \mathrel{::=}&
  n
  \mid b
  \mid x\texttt{(}y\texttt{:}\,\tau\texttt{)}\texttt{:}\,\tau' \mathrel{\texttt{=}\!\texttt{>}} e_1
  \\
  \text{expressions} & e& \mathrel{::=}&
  x
  \mid\mathbf{const}\;x\;\texttt{=}\;e_1\texttt{;}\;e_2
  \\
  & & \mid& n
  \mid\mathop{\mathit{uop}} e_1
  \mid e_1 \mathbin{\mathit{bop}} e_2
  \\
  & & \mid& b
  \mid e_1\;\texttt{?}\;e_2\;\texttt{:}\;e_3
  \\
  & & \mid& x\texttt{(}y\texttt{:}\,\tau\texttt{)}\texttt{:}\,\tau' \mathrel{\texttt{=}\!\texttt{>}} e_1
  \mid e_1\texttt{(}e_2\texttt{)}
  \\
  \text{unary operators} & \mathit{uop}& \mathrel{::=}& \texttt{-}\mid\texttt{!}
  \\
  \text{binary operators} & \mathit{bop}& \mathrel{::=}& \texttt{+}\mid\texttt{*}\mid\texttt{===}
\end{array}
$$

Figure 1: Syntax of TypeScripty with recursive functions and limited
arithmetic-logical expressions.

We give a Scala representation of the abstract syntax, along with an
implementation of computing the free variables of an expression and
determining an expression is closed:

In [ ]:
trait Typ                                                // t
case object TNumber extends Typ                          // t ::= number
case object TBool extends Typ                            // t ::= bool
case class TFun(yt: (String,Typ), tret: Typ) extends Typ // t ::= (y: t) => tret

trait Expr                                                       // e
case class Var(x: String) extends Expr                           // e ::= x
case class ConstDecl(x: String, e1: Expr, e2: Expr) extends Expr // e ::= const x = e1; e2

case class N(n: Double) extends Expr                         // e ::= n
case class Unary(uop: Uop, e1: Expr) extends Expr            // e ::= uop e1
case class Binary(bop: Bop, e1: Expr, e2: Expr) extends Expr // e ::= e1 bop b2

case class B(b: Boolean) extends Expr                    // e ::= b
case class If(e1: Expr, e2: Expr, e3: Expr) extends Expr // e ::= e1 ? e2 : e3

case class Fun(x: String, yt: (String, Typ), tret: Typ, e1: Expr) extends Expr // e ::= x(y: t): tret => e1
case class Call(e1: Expr, e2: Expr) extends Expr                               // e ::= e1(e2)

trait Uop                   // uop
case object Neg extends Uop // uop ::= -
case object Not extends Uop // uop ::= !

trait Bop                     // bop
case object Plus extends Bop  // bop ::= *
case object Times extends Bop // bop ::= *
case object Eq extends Bop    // bop ::= ===

def freeVars(e: Expr): Set[String] = e match {
  case Var(x) => Set(x)
  case ConstDecl(x, e1, e2) => freeVars(e1) | (freeVars(e2) - x)
  case N(_) | B(_) => Set.empty
  case Unary(_, e1) => freeVars(e1)
  case Binary(_, e1, e2) => freeVars(e1) | freeVars(e2)
  case If(e1, e2, e3) => freeVars(e1) | freeVars(e2) | freeVars(e3)
  case Fun(x, (y, _), _, e1) => freeVars(e1) - x - y
  case Call(e1, e2) => freeVars(e1) | freeVars(e2)
}

def isClosed(e: Expr): Boolean = freeVars(e).isEmpty

### Static Type Checking

The typing judgment $\Gamma \vdash e : \tau$ says, “In typing
environment $\Gamma$, expression $e$ has type $\tau$” where the typing
environment $\Gamma\mathrel{::=}\cdot\mid\Gamma, x : \tau$ is a finite
map from variables to types, assigning types to the free variables of
$e$. We give the expected typing rules that we have seen before,
restricted to this simpler language:

$\fbox{$\Gamma \vdash e : \tau$}$

$\inferrule[TypeVar]{
}{
  \Gamma \vdash x : \Gamma(x)
}$

$\inferrule[TypeConstDecl]{
  \Gamma \vdash e_1 : \tau_1
  \and
  \Gamma, x : \tau_1 \vdash e_2 : \tau_2
}{
  \Gamma \vdash \mathbf{const}\;x\;\texttt{=}\;e_1\texttt{;}\;e_2 : \tau_2
}$

$\inferrule[TypeNumber]{
}{
  \Gamma \vdash n : \texttt{number}
}$

$\inferrule[TypeNeg]{
  \Gamma \vdash e_1 : \texttt{number}
}{
  \Gamma \vdash \mathop{\texttt{-}} e_1 : \texttt{number}
}$

$\inferrule[TypeArith]{
  \Gamma \vdash e_1 : \texttt{number}
  \and
  \Gamma \vdash e_2 : \texttt{number}
  \and
  \mathit{bop}\in \left\{  \texttt{+}, \texttt{*} \right\}
}{
  \Gamma \vdash e_1 \mathbin{\mathit{bop}}e_2 : \texttt{number}
}$

$\inferrule[TypeBool]{
}{
  \Gamma \vdash b : \texttt{bool}
}$

$\inferrule[TypeEq]{
  \Gamma \vdash e_1 : \tau
  \and
  \Gamma \vdash e_2 : \tau
}{
  \Gamma \vdash e_1 \mathbin{\texttt{===}} e_2 : \texttt{bool}
}$

$\inferrule[TypeIf]{
  \Gamma \vdash e_1 : \texttt{bool}
  \and
  \Gamma \vdash e_2 : \tau
  \and
  \Gamma \vdash e_3 : \tau
}{
  \Gamma \vdash e_1\;\texttt{?}\;e_2\;\texttt{:}\;e_3 : \tau
}$

$\inferrule[TypeFunctionRec]{
  \Gamma, x :  \texttt{(}y\texttt{:}\,\tau\texttt{)} \mathrel{\texttt{=}\!\texttt{>}} \tau' , y : \tau \vdash e' : \tau'
}{
  \Gamma \vdash  x\texttt{(}y\texttt{:}\,\tau\texttt{)}\texttt{:}\,\tau' \mathrel{\texttt{=}\!\texttt{>}} e'  :  \texttt{(}y\texttt{:}\,\tau\texttt{)} \mathrel{\texttt{=}\!\texttt{>}} \tau' 
}$

$\inferrule[TypeCall]{
  \Gamma \vdash e_1 :  \texttt{(}y\texttt{:}\,\tau\texttt{)} \mathrel{\texttt{=}\!\texttt{>}} \tau' 
  \and
  \Gamma \vdash e_2 :  \tau
}{
  \Gamma \vdash e_1\texttt{(}e_2\texttt{)} : \tau'
}$

Figure 2: Typing of TypeScripty with recursive functions and limited
arithmetic-logical expressions.

## Error Effects

As we have seen, the most direct way to implement a type checker
following the typing judgment $\Gamma \vdash e : \tau$ is a function
`hastype: (Map[String, Typ], Expr) => Typ` that takes an typing
environment $\Gamma$ represented by a `Map[String, Typ]` and an
expression $e$ represented by a `Expr` and returns a type $\tau$
represented by a `Typ`:

``` scala
def hastype(tenv: Map[String, Typ], e: Expr): Typ = ???
```

However, with this function signature for `hastype`, we necessarily have
to throw an exception or crash to indicate that an expression $e$ is
ill-typed. An expression is ill-typed when there are no rules that allow
us to derive a judgment $\Gamma \vdash e : \tau$ for any $\tau$ and
$\Gamma$.

In this exercise, we implement a version of `hastype` that makes
explicit the possibility of a type error.

### Type-Error Result

We first extend our typing judgment form $\Gamma \vdash e : r$ where a
type-checker result $r$ is either a type $\tau$ or a type-error result
$\mathit{typerr}$:

<span class="text">type-checker result</span>
$\quad  r  \mathrel{::=}  \mathit{typerr}  \mid  \tau$

<span class="text">type-error result</span>
$\quad  \mathit{typerr}  \mathrel{::=}  \mathop{\mathsf{typerr}}(e : \tau)$

This extended judgment form makes explicit when an expression is
ill-typed. A type-error result

$$
\mathop{\mathsf{typerr}}(e : \tau)
$$

includes an expression $e$ with its inferred type $\tau$ but is used in
a context another type is expected. We can consider such a type-error
result being used to give a descriptive error message to the programmer.

We now give some rules that introduce type-error results and propagates
them:

$\fbox{$\Gamma \vdash e : r$}$

$\inferrule[TypeErrorNeg]{
  \Gamma \vdash e_1 : \tau_1
  \and
  \tau_1 \neq \texttt{number}
}{
  \Gamma \vdash \mathop{\texttt{-}} e_1 :  \mathop{\mathsf{typerr}}(e_1 : \tau_1) 
}$

$\inferrule[PropagateNeg]{
  \Gamma \vdash e_1 :  \mathit{typerr}
}{
  \Gamma \vdash \mathop{\texttt{-}} e_1 :  \mathit{typerr}
}$

$\inferrule[TypeErrorEq]{
  \Gamma \vdash e_1 : \tau_1
  \and
  \Gamma \vdash e_2 : \tau_2
  \and
  \tau_1 \neq \tau_2
}{
  \Gamma \vdash e_1 \mathbin{\texttt{===}} e_2 :  \mathop{\mathsf{typerr}}(e_2 : \tau_2) 
}$

$\inferrule[PropagateEq1]{
  \Gamma \vdash e_1 :  \mathit{typerr}
}{
  \Gamma \vdash e_1 \mathbin{\texttt{===}} e_2 :  \mathit{typerr}
}$

$\inferrule[PropagateEq2]{
  \Gamma \vdash e_2 :  \mathit{typerr}
}{
  \Gamma \vdash e_1 \mathbin{\texttt{===}} e_2 :  \mathit{typerr}
}$

Observe that the $\TirName{TypeErrorEq}$ blames $e_2$: it infers the
type $\tau_1$ for $e_1$ and then expects that $e_2$ has type $\tau_1$.

<span class="theorem-title">**Exercise 1 (4 points)**</span> Define the
$\TirName{TypeErrorNot}$ rule that introduces a type-error result when
the boolean negation expression $\mathop{\texttt{!}} e_1$ is ill-typed:

**Edit this cell:**

YOUR ANSWER HERE

#### Notes

-   Hint: Use $\TirName{TypeErrorNeg}$ as a model.
-   You may give the rule in LaTeX math or as plain text (ascii art)
    approximating the math rendering. For example,

<!-- -->

    TypeErrorNeg
    Gamma |- e1 : tau1  tau1 != number
    ------------------------------------------
    Gamma |- -e1 : typerr(e1 : tau1 != number)

The LaTeX code for the rendered $\TirName{TypeErrorNeg}$ rule above is
as follows:

``` tex
\inferrule[TypeErrorNeg]{
  \Gamma \vdash e_1 : \tau_1
  \and
  \tau_1 \neq \texttt{number}
}{
  \Gamma \vdash \mathop{\texttt{-}} e_1 :  \mathop{\mathsf{typerr}}(e_1 : \tau_1 \neq \texttt{number})
}
```

<span class="theorem-title">**Exercise 2 (8 points)**</span> Define
**two** $\TirName{TypeErrorIf}$ rules that introduces a type-error
result when the if-then-else expression
$e_1\;\texttt{?}\;e_2\;\texttt{:}\;e_3$ is ill-typed:

**Edit this cell:**

YOUR ANSWER HERE

#### Notes

-   Hint: Use $\TirName{TypeErrorNeg}$ and $\TirName{TypeErrorEq}$ as a
    model for the two rules, respectively.

### Implementation

We represent a type-error result $r$ as an
`Either[StaticTypeError, Typ]`.

<span class="theorem-title">**Exercise 3 (26 points)**</span> Complete
the following implementation of

`hastype: (Map[String, Typ], Expr) => Either[StaticTypeError, Typ]`

corresponding to the judgment form $\Gamma \vdash e : r$.

**Edit this cell:**

In [ ]:
case class StaticTypeError(tbad: Typ, esub: Expr, e: Expr) {
  override def toString = s"StaticTypeError: invalid type $tbad for sub-expression $esub in $e"
}

def hastype(tenv: Map[String, Typ], e: Expr): Either[StaticTypeError, Typ] = {

  def err[T](esub: Expr, tgot: Typ): Either[StaticTypeError, T] =
    Left(StaticTypeError(tgot, esub, e))

  def typecheck(tenv: Map[String, Typ], e: Expr, tshould: Typ): Either[StaticTypeError, Unit] =
    hastype(tenv, e) flatMap { tgot =>
      if (tgot == tshould)
        ???
      else err(e, tgot)
    }

  e match {
    case Var(x) => Right(tenv(x))
    case ConstDecl(x, e1, e2) =>
      ???
    case N(_) => Right(TNumber)
    case B(_) => Right(TBool)
    case Unary(Neg, e1) => typecheck(tenv, e1, TNumber) map { _ => TNumber }
    case Unary(Not, e1) =>
      ???
    case Binary(Plus|Times, e1, e2) =>
      ???
    case Binary(Eq, e1, e2) =>
      hastype(tenv, e1) flatMap { t1 =>
        hastype(tenv, e2) flatMap { t2 =>
          if (t1 == t2) Right(TBool) else err(e2, t2)
        }
      }
    case If(e1, e2, e3) => typecheck(tenv, e1, TBool) flatMap { _ =>
      ???
    }
    case Fun(x, yt @ (y, t), tret, e1) => {
      ???
    }
    case Call(e1, e2) =>
      ???
  }
}

def inferType(e: Expr): Either[StaticTypeError, Typ] = {
  require(isClosed(e), s"$e should be closed")
  hastype(Map.empty, e)
}

#### Notes

-   The `err` helper function is simply a shortcut to construct
    `StaticTypeError` with the input expression `e`.
-   The `typecheck` helper function implements a common functionality
    where we want to call `hastype` recursively with a sub-expression to
    infer a type `tgot` and check it is equal to an expected type
    `tshould`. If `tgot != tshould`, we return an `err`; otherwise, we
    return success.
-   Hint: Start by implementing the $\TirName{Type}$ rules from
    <a href="#fig-typescripty-minrecfun-typing"
    class="quarto-xref">Figure 2</a> without worrying about type errors.
    Then, add $\TirName{TypeError}$ and $\TirName{Propagate}$ cases.
-   Note that not all the $\TirName{TypeError}$ and
    $\TirName{Propagate}$ rules are given, but they follow the patterns
    given above. If you get stuck to implement the code for
    $\TirName{TypeError}$ for a particular construct, try writing out
    the rule.
-   It is fine to initially pattern match on
    `Either[StaticTypeError, Expr]` values that result from calls to
    `hastype` and `typecheck` (i.e., for `Left` and `Right` values).
    However, challenge yourself to replace them with `map` or `flatMap`
    calls to minimize your typing and opportunities for bugs to creep
    in. It is possible to replace all pattern matches for `Left` and
    `Right` values with calls to `map` and `flatMap` calls.
-   Use the given cases (e.g., $\TirName{TypeNeg}$,
    $\TirName{TypeErrorNeg}$, $\TirName{PropagateNeg}$) as a model.
    Identify how $\TirName{TypeNeg}$, $\TirName{TypeErrorNeg}$, and
    $\TirName{PropagateNeg}$ manifests in the code.
-   For accelerated students, you can make the code even more compact
    (and arguably more readable) by replacing `flatMap` and `map` calls
    with `for`-`yield` expressions. Note that not all cases can be
    implemented with `for`-`yield` expressions.

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val e1 = Binary(Plus, N(1), N(2)) 
  val e2 = Binary(Times, Var("a"), N(3))

  val e3 = ConstDecl("a", e1, e2)
  "hastype/ConstDecl" should "TypeConstDecl" in {
    assertResult(Right(TNumber)) {inferType(e1) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateConstDecl1" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(ConstDecl("a", bade, N(1))) }
  }

  it should "PropagateConstDecl2" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(ConstDecl("a", N(1), bade)) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val e1 = Unary(Not, B(true))
  "hastype/Unary/Not" should "TypeNot" in {
    assertResult(Right(TBool)) {inferType(e1) }
  }

  val e2 = Unary(Not, N(1))
  it should "TypeErrorNot" in {
    assertResult(Left(StaticTypeError(TNumber, N(1), e2))) {inferType(e2) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateNot" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(Unary(Not, bade)) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val e1 = Binary(Plus, N(1), N(2))
  "hastype/Binary/Plus|Times" should "TypeArith" in {
    assertResult(Right(TNumber)) { inferType( Binary(Times, e1, N(2)) ) }
  }

  val e2 = Binary(Plus, B(true), e1)
  it should "TypeErrorArith" in {
    assertResult(Left(StaticTypeError(TBool, B(true), e2))) { inferType(e2) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateArith" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(Binary(Plus, bade, N(3))) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val e1 = Binary(Plus, N(1), N(2))
  "hastype/If" should "TypeIf" in {
    assertResult(Right(TNumber)) { inferType( If(B(true), e1, N(3)) ) }
  }

  val e2 = If(N(1), e1, e1)
  it should "TypeErrorIfGuard" in {
    assertResult(Left(StaticTypeError(TNumber, N(1), e2))) { inferType(e2) }
  }

  val e3 = If(B(false), B(true), N(1))
  it should "TypeErrorIfBody" in {
    assertResult(Left(StaticTypeError(TNumber, N(1), e3))) { inferType(e3) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateIf" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(If(B(true), bade, N(3))) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val e1 = Binary(Times, Var("i"), N(3))
  def fun(e: Expr) = Fun("f", ("i", TNumber), TNumber, e)
  "hastype/Fun" should "TypeFun" in {
    assertResult(Right(TFun(("i", TNumber), TNumber))) { inferType(fun(e1)) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateFun" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(fun(bade)) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val e1 = Binary(Times, Var("i"), Call(Var("f"), N(3)))
  def fun(e: Expr) = Fun("f", ("i", TNumber), TNumber, e)
  val e2 = 
  "hastype/Call" should "TypeCall" in {
    assertResult(Right(TNumber)) { inferType(Call(fun(e1), N(10))) }
  }

  val e3 = Call(B(true), N(1))
  it should "TypeErrorCall" in {
    assertResult(Left(StaticTypeError(TBool, B(true), e3))) { inferType(e3) }
  }

  val badesub = B(true)
  val bade = Binary(Plus, N(1), badesub)
  it should "PropagateCall2" in {
    assertResult(Left(StaticTypeError(TBool, badesub, bade))) { inferType(Call(fun(e1),bade)) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  val factorial = Fun("fact", ("n", TNumber), TNumber,
    If(Binary(Eq, Var("n"), N(1)),
    N(1),
    Binary(Times, Var("n"), Call(Var("fact"), Binary(Plus, Var("n"), N(-1)))))
  )
  "hastype" should "type check factorial" in {
    assertResult(Right(TNumber)) { inferType(Call(factorial, N(4))) }
  }
}, 2)

## Mutation Effects

### Defining Generic `DoWith` Methods

Recall the idea of encapulating a state transforming function
`S => (S, A)` as a collection-like data type
(**?@sec-encapsulating-mutation-effects**), which we call a
`DoWith[S, A]`:

In [ ]:
sealed class DoWith[S, A] private (doer: S => (S, A)) {
  def map[B](f: A => B): DoWith[S, B] = new DoWith[S, B]({
    (s: S) => {
      val (s_, a) = doer(s)
      (s_, f(a))
    }
  })

  def flatMap[B](f: A => DoWith[S, B]): DoWith[S, B] = new DoWith[S, B]({
    (s: S) => {
      val (s_, a) = doer(s)
      f(a)(s_)
    }
  })

  def apply(s: S): (S, A) = doer(s)
}

object DoWith {
  def doget[S]: DoWith[S, S] = new DoWith[S, S]({ s => (s, s) })
  def doput[S](s: S): DoWith[S, Unit] = new DoWith[S, Unit]({ _ => (s, ()) })
}

import DoWith._

Consider a `DoWith[S,A]` as a “collection” holding an `A` somewhat like
`List[A]` or `Option[A]`. While `List[A]` encapsulates a sequence of
elements of type `A` elements and `Option[A]` encapsulates an optional
`A`, a `DoWith[S, A]` encapsulates a *computation* that results in an
`A`, using an input-output state `S` (i.e., a `S => (S, A)` function).
It is collection-like because it also has `map` and `flatMap` methods to
transform that computation.

We also define two functions for a client to create a `DoWith[S, A]`
encapsulating particular `S => (S, A)`{.scala functions}:

-   The `doget[S]` method constructs a `DoWith[S, S]` that makes the
    “current” state the result (i.e., `s => (s, s)`). Intuitively, it
    “gets” the state.
-   The `doput[S](s: S)` method constructs a `DoWith[S, Unit]` that
    makes the given state `s` the “current” state (i.e.,
    `_ => (s, ())`). Intuitively, it “puts” `s` into the state.

We require the client to create `DoWith[S, A]` values using only the
`doget` and `doput` constructors.

<span class="theorem-title">**Exercise 4 (4 points)**</span> Read the
implementations `map[B]` and `flatMap[B]` methods of `DoWith[S, A]`
above. Explain how they transform the encapsulated `doer: S => (S, A)`
function. Compare and contrast them—what do they do that is same, and
what is the key difference between them?

**Edit this cell:**

YOUR ANSWER HERE

As a `DoWith[S, A]` encapsulates any function of type `S => (S, A)`,
there are other commonly needed functions of this type. We implement
methods for constructing `DoWith[S, A]` encapsulating two more
commonly-needed computations in terms of `doget` and `doput`.

-   The `doreturn[S, A](a: A)` method constructs a `DoWith[S, A]` that
    leaves the “current” state as-is and returns the given result `a`
    (i.e., `s => (s, a)`). It technically does not need to be given in
    the library, as it can be defined in terms of `doget` and `map`.
-   The `domodify[S](f: S => S)` method constructs a `DoWith[S, Unit]`
    that “modifies” the state using the given function `f: S => S`
    (i.e., `s => (f(s), ())`). It technically does not need to be given
    in the library, as it can be defined in terms of `doget`, `doput`,
    and `flatMap`.

<span class="theorem-title">**Exercise 5 (4 points)**</span> Implement
`doreturn[S, A](a: A)` that creates a `DoWith[S, A]` that encapsulates
the computation `(s: S) => (s, a)` using `doget`, `doput`, `map`, and/or
`flatMap`.

**Edit this cell:**

In [ ]:
def doreturn[S, A](a: A): DoWith[S, A] =
  ???

#### Tests

In [ ]:
test(new AnyFlatSpec {
  "doreturn" should "Test1" in { assertResult( (0,1) ) { doreturn(1)(0) }}
  it should "Test2" in { assertResult( (2,"test") ) { doreturn("test")(2) }}
  it should "Test3" in { assertResult( (2,List(1,2,3)) ) { doreturn(List(1,2,3))(2) }}
}, 4)

<span class="theorem-title">**Exercise 6 (4 points)**</span> Implement
`domodify[S](f: S => S)` that creates a `DoWith[S, Unit]` that
encapsulates the computation `(s: S) => (f(s), ())` using `doget`,
`doput`, `map`, and/or `flatMap`.

**Edit this cell:**

In [ ]:
def domodify[S](f: S => S): DoWith[S, Unit] = 
  ???

#### Notes

-   Hint: To define `doreturn` and `domodify`, it might help first to do
    type-directed programming where you ignore what a `DoWith[S, A]`
    actually is and think of it as `Either[S, A]` or abstractly as
    `M[S, A]` for an unknown type constructor `M`. There only limited
    things you can do by composing `doget`, `doput`, `map`, and
    `flatMap`.

#### Tests

In [ ]:
test(new AnyFlatSpec {
  "domodify" should "Test1" in { assertResult( (6,()) ) { domodify({a:Int => 2*a})(3) }}
  it should "Test2" in { assertResult( ("testtest",()) ) { domodify({x:String => x+x})("test") }}
  it should "Test3" in { assertResult( (List(1,2,3,4),()) ) { domodify({l:List[Int] => l:+4})(List(1,2,3))}}
}, 4)

### Renaming Bound Variables

Recall that with static scoping, we see expressions as being equivalent
up to the renaming of bound variables
(**?@sec-renaming-bound-variables**). For example, we see the following
two expressions as being equivalent for the purposes of the language
implementation:

``` js
const four = (2 + 2); (four + four)
```

``` js
const x = (2 + 2); (x + x)
```

even though the concrete syntax for the human user is different in their
choice of variable names.

While being able to choose variable names that shadows another variable
is important for the human user, we have seen that clashing variable
names adds complexity to the language implementation. For example,
mishandling of clashing variable names results in accidental dynamic
scoping (**?@sec-dynamic-scoping**) and substitution has to account for
variable shadowing (**?@sec-substitution**).

Observing that renaming bound variables preserves meaning for the
language implementation, one approach for simplifying the handling of
variable scope is to implement a *lowering* pass that renames bound
variables in a consistent manner so that variable names are globally
unique. For example, we could lower the following expression

``` js
const a = ((a) => a)(1) + ((a) => a)(2); a
```

by renaming bound variables to, for example,

``` js
const a_0 = ((a_1) => a_1)(1) + ((a_2) => a_2)(2); a_0
```

We will implement the above lowering pass in a few steps.

<span class="theorem-title">**Exercise 7 (4 points)**</span> Write a
function `freshVar(x: String)` that returns a function of type
`Int => (Int, String)`, which takes a current index-state `i: Int` and
returns the pair of the next index state `i + 1` and the string `x` with
the index appended and separated by `"_"` (specifically,
`s"${x}_${i}"`):

**Edit this cell:**

In [ ]:
def freshVar(x: String): Int => (Int, String) =
  { i =>
      ???
  }

#### Tests

In [ ]:
test(new AnyFlatSpec with Matchers with ScalaCheckPropertyChecks {
  "freshVar" should "satisfy the property" in {
    forAll ("x", "i") { (x: String, i: Int) =>
      freshVar(x)(i) should be (i + 1, s"${x}_${i}")
    }
  }
}, 4)

Aside: Writing lots of tests is painful. In the above, we use an idea
called *property-based testing* to make it less painful and more
effective. A property-based testing library enables specifying a
property on inputs like in the above for `x: String` and `i: Int`, and
the library will choose lots of random inputs with which to check the
property. Importantly, the library also enables the client to specify
how to generate inputs (e.g., what range, what distribution), though we
do not do anything special to control the input generation in the above.

<span class="theorem-title">**Exercise 8 (4 points)**</span> Write a
function `freshVarWith(x: String): DoWith[Int, String]` that behaves
like `freshVar(x: String): Int => (Int, String)` in
<a href="#exr-freshVar" class="quarto-xref">Exercise 7</a>.

**Edit this cell:**

In [ ]:
def freshVarWith(x: String): DoWith[Int, String] =
  ???

#### Notes

-   Hint: Remember that `DoWith[Int, String]` encapsulates a function of
    type `Int => (Int, String)`. Use `freshVar` as a guide to creating
    the `DoWith[Int, String]` using `doget`, `doput`, `map`, and/or
    `flatMap`.
-   For accelerated students, you may try to use a `for`-`yield`
    expression to replace your `map` and `flatMap` calls.

A lowering pass is a transformation function from `Expr => Expr`. To
rename bound variables in a consistent manner, we need an enviroment to
remember what the name should be for a free variable use in the input
`Expr`. Thus, we need to define a helper function
`rename(env: Map[String, String], e: Expr)` that takes as input an
`env: Map[String, String]` that maps original names to new names for the
free variable uses in `e`.

We also need a way to choose how to rename bound variables so that they
are unique. The client of `rename` might want to rename variables
uniquely in different ways (e.g., using an integer counter, using the
original name with an integer counter). Thus, we add an additional
callback parameter

`fresh: String => DoWith[S, String]`

for the client to specify how they want to specify the fresh name given
the original name where they can choose a state type `S` to use through
renaming.

#### Tests

In [ ]:
test(new AnyFlatSpec with Matchers with ScalaCheckPropertyChecks {
  "freshVarWith" should "satisfy the property" in {
    forAll ("x", "i") { (x: String, i: Int) =>
      freshVarWith(x)(i) should be (i + 1, s"${x}_${i}")
    }
  }
}, 4)

<span class="theorem-title">**Exercise 9 (24 points)**</span> Implement
a function `rename` that renames variable names in `e` consistently
using the given callback to `fresh` to generate fresh names for bound
variables and `env` to rename free variable uses.

**Edit this cell:**

In [ ]:
def rename[S](env: Map[String, String], e: Expr)(fresh: String => DoWith[S, String]): DoWith[S, Expr] = {
  def rename(env: Map[String, String], e: Expr): DoWith[S, Expr] = {
    def r(e: Expr) = rename(env, e)
    e match {
      case N(_) => doreturn(e)
      case B(_) =>
        ???

      case Var(x) =>
        ???

      case ConstDecl(x, e1, e2) => fresh(x) flatMap { x_ =>
        ???
      }

      case Unary(uop, e1) =>
        ???

      case Binary(bop, e1, e2) =>
        ???

      case If(e1, e2, e3) =>
        ???

      case Fun(x, (y, t), tret, e1) =>
        ???

      case Call(e1, e2) =>
        r(e1) flatMap { e1 =>
          r(e2) map { e2 => Call(e1, e2) }
        }
    }
  }
  rename(env, e)
}

#### Notes

-   Hint: Use the given cases for `N` and `Call` as models. The only
    methods for manipulating `DoWith` objects needed in this exercise
    are `doreturn`, `map`, and `flatMap`.
-   **Accelerated**: You may try to use `for`-`yield` expressions to
    replace your `map` and `flatMap` calls.

### Test

In [ ]:
test(new AnyFlatSpec {
 "rename" should "rename in a DoWith" in {
    val e1 = ConstDecl("a", N(1), Var("a"))
    val e1p = ConstDecl("aa", N(1), Var("aa"))

    assertResult( (1, e1p) ) {
      rename(Map.empty, e1){ x => domodify[Int](n => n + 1) map { _ => x + x } }(0)
    }
  }
 }, 4)

In [ ]:
test(new AnyFlatSpec {
  def fresh(s: String): DoWith[Int,String] = doget flatMap { i => doput(i + 1) map { _ => s"x${i}" } }
  def renv(env: Map[String, String], e: Expr): (Int, Expr) = rename(env, e)(fresh)(0)
  def r(e: Expr) = renv(Map.empty, e)

  "rename/B" should "RenameB" in {
    assertResult( (0, B(true)) ) { r(B(true)) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  def fresh(s: String): DoWith[Int,String] = doget flatMap { i => doput(i + 1) map { _ => s"x${i}" } }
  def renv(env: Map[String, String], e: Expr): (Int, Expr) = rename(env, e)(fresh)(0)
  def r(e: Expr) = renv(Map.empty, e)

  "rename/Var" should "RenameVar" in {
    assertResult( (0, Var("y")) ) { renv(Map("x" -> "y"), Var("x")) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  def fresh(s: String): DoWith[Int,String] = doget flatMap { i => doput(i + 1) map { _ => s"x${i}" } }
  def renv(env: Map[String, String], e: Expr): (Int, Expr) = rename(env, e)(fresh)(0)
  def r(e: Expr) = renv(Map.empty, e)

  def e(x: String) = ConstDecl(x, N(1), Var(x))
  "rename/ConstDecl" should "RenameConstDecl" in {
    assertResult( (1, e("x0")) ) { r(e("foo")) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  def fresh(s: String): DoWith[Int,String] = doget flatMap { i => doput(i + 1) map { _ => s"x${i}" } }
  def renv(env: Map[String, String], e: Expr): (Int, Expr) = rename(env, e)(fresh)(0)
  def r(e: Expr) = renv(Map.empty, e)

  def e(f: String, y: String) = Fun(f, (y, TNumber), TNumber, Call(Var("f"), Var("y")))
  "rename/Fun" should "RenameFun" in {
    assertResult( (2, e("x0", "x1")) ) { r(e("foo", "bar")) }
  }
}, 4)

In [ ]:
test(new AnyFlatSpec {
  def fresh(s: String): DoWith[Int,String] = doget flatMap { i => doput(i + 1) map { _ => s"x${i}" } }
  def renv(env: Map[String, String], e: Expr): (Int, Expr) = rename(env, e)(fresh)(0)
  def r(e: Expr) = renv(Map.empty, e)

  def factorial(fact: String, n: String) = Fun(fact, (n, TNumber), TNumber,
    If(Binary(Eq, Var(n), N(1)),
    N(1),
    Binary(Times, Var(n), Call(Var(fact), Binary(Plus, Var(n), N(-1)))))
  )

  "rename" should "rename bound variables consistently for factorial" in {
    assertResult( (2, factorial("x0", "x1")) ) { r(factorial("fact", "n")) }
  }

  it should "rename with a different fresh" in {
    assertResult( (4, factorial("fact0", "n2")) ) {
      val e = factorial("fact", "n")
      rename[Int](Map.empty, e)(x => doget flatMap { i => doput(i + 2) map { _ => s"${x}${i}" } })(0)
    }
  }
}, 4)

<span class="theorem-title">**Exercise 10 (4 points)**</span> Finally,
implement the lowering-transformation function `uniquify: Expr => Expr`
on closed expressions using a call to `rename` to rename bound variables
consistently with the `freshVarWith: String => DoWith[Int, String]`
function defined above. Start the `Int` counter-state at `0`.

**Edit this cell:**

In [ ]:
def uniquify(e: Expr): Expr = {
  require(isClosed(e), s"$e should be closed")
  ???
}

#### Notes

-   Hint: The `rename` and `freshVarWith` functions constructs `DoWith`
    objects. The `uniquify` finally uses a `DoWith[Int, String]` object.
    How? By calling it with the initial state `0`.

#### Test

In [ ]:
test(new AnyFlatSpec with Matchers with ScalaCheckPropertyChecks {
  def idfun(f: String, y: String) =
    Fun(f, (y, TNumber), TNumber, Var(y))
  def aconst(x1: String, x2: String, x3: String, x4: String, x5: String) =
    ConstDecl(x1, Binary(Plus, Call(idfun(x2, x3), N(1)), Call(idfun(x4, x5), N(2))), Var(x1))
  def nm(x: String, i: Int) = s"${x}_${i}"

  "uniquify" should "work on the example" in {
    forAll { (x1: String, x2: String, x3: String, x4: String, x5: String) =>
      uniquify(aconst(x1, x2, x3, x4, x5)) should be {
        aconst(nm(x1, 0), nm(x2, 1), nm(x3, 2), nm(x4, 3), nm(x5, 4))
      }
    }
  }

  val e = ConstDecl("a", ConstDecl("a", N(1), Var("a")), ConstDecl("b",N(2), Var("b")))
  val exp = ConstDecl("a_0", ConstDecl("a_1", N(1), Var("a_1")), ConstDecl("b_2", N(2), Var("b_2")))
  it should "work another example" in {
    assertResult(exp) { uniquify(e) }
  }
}, 4)

### `DoWith` with Collections

Recall the higher-order function `map` we considered previously for
`List`s.

In [ ]:
def map[A, B](l: List[A])(f: A => B): List[B] =
  l.foldRight[List[B]](
    Nil
  ) { (h, acc) =>
    f(h) :: acc
  }

We may want to use `DoWith` to encapsulate some stateful computation
with other data structures, like `List`s. For example, we want a version
of `map` that can take a stateful callback:

<span class="theorem-title">**Exercise 11 (4 points)**</span> Implement
a function `mapWith` for `List` that takes a stateful callback function
`f: A => DoWith[S, B]`:

**Edit this cell:**

In [ ]:
def mapWith[S, A, B](l: List[A])(f: A => DoWith[S, B]): DoWith[S, List[B]] =
  l.foldRight[DoWith[S, List[B]]](
    ???
  ) { (h, acc) =>
    ???
  }

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val l = List(1, 2, 3, 4, 5)
  val r1 = l.map { i => i + 1 }

  def dowith1[W]: DoWith[W,List[Int]] = mapWith(l) { i: Int => doreturn(i + 1) }
      
  "mapWith" should "Test1" in { assertResult((true,r1)) { dowith1(true) } }
  it should "Test2" in {assertResult((42,r1)) { dowith1(42) }}
  it should "Test3" in {assertResult((2 * l.length + 1, r1)) {
        val dw: DoWith[Int,List[Int]] = mapWith(l) { i: Int =>
          domodify[Int](s => s + 2) map { _ => i + 1 }
        }
        dw(1)
      }}
}, 4)

Also recall the `mapFirst` function we defined previously that replaces
the first element in `l` where `f` returns `Some(a)` with `a`:

In [ ]:
def mapFirst[A](l: List[A])(f: A => Option[A]): List[A] = l match {
  case Nil => Nil
  case h :: t =>
    f(h) match {
      case None => h :: mapFirst(t)(f)
      case Some(h) => h :: t
    }
}

<span class="theorem-title">**Exercise 12 (4 points)**</span> Implement
a function `mapFirstWith` for `List` that instead takes a stateful
callback function `f: A => Option[DoWith[S, B]]`:

**Edit this cell:**

In [ ]:
def mapFirstWith[S, A](l: List[A])(f: A => Option[DoWith[S, A]]): DoWith[S, List[A]] = l match {
  case Nil =>
    ???
  case h :: t =>
    ???
}

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val l = List(1, 2, -3, 4, -5)
  val r = List(1, 2, 3, 4, -5)
  def dowith[W]: DoWith[W,List[Int]] = mapFirstWith(l) { (i: Int) => if (i < 0) Some(doreturn(-i)) else None }
      
  val w1 = true
  val w2 = 10
  val w3 = "test_str"
  val w4 = List
      
  "mapFirstWith" should "Test1" in { assertResult((w1,r)){ dowith(w1) } }
  it should "Test2" in {assertResult((w2,r)) { dowith(w2) }}
  it should "Test3" in {assertResult((w3,r)) { dowith(w3) }}
  it should "Test4" in {assertResult((w4,r)) { dowith(w4) }}
}, 4)